## Project 2 - ETL

### Import Dependencies

In [1]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import inspect
from datetime import datetime, timedelta

### Extract CSVs into DataFrames

In [2]:
application_record_file = "Resources/application_record.csv"
application_record_df = pd.read_csv(application_record_file)
application_record_df.head()

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0


In [3]:
credit_record_file = "Resources/credit_record.csv"
credit_record_df = pd.read_csv(credit_record_file)
credit_record_df.head()

,ID,MONTHS_BALANCE,STATUS
0,5001711,0,X
1,5001711,-1,0
2,5001711,-2,0
3,5001711,-3,0
4,5001712,0,C


### Transform Application Record DataFrame (application_record_df)

In [4]:
# Remove the duplicate rows from the DataFrame
application_record_df = application_record_df.drop_duplicates()

#### Clean Days_Birth column

In [5]:
def subtract_days(num_days):
  # Get the current date
  today = datetime.now()

  # Always create a timedelta object with a positive number of days
  num_days_ago = timedelta(days=abs(num_days))

  # Subtract the timedelta from the current date to get the date num_days ago
  num_days_ago_date = today - num_days_ago

  # Return the date num_days ago as a string in the desired format
  return num_days_ago_date.strftime("%Y-%m-%d")

# Use the apply() method to apply the subtract_days() function to every value in the 'days_to_subtract' column
application_record_df['DAYS_BIRTH'] = application_record_df['DAYS_BIRTH'].apply(subtract_days)

# Print the resulting DataFrame
application_record_df.head()

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,1990-02-05,-4542,1,1,0,0,NaN,2.0
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,1990-02-05,-4542,1,1,0,0,NaN,2.0
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,1964-03-04,-1134,1,0,0,0,Security staff,2.0
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,1970-08-24,-3051,1,0,1,1,Sales staff,1.0
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,1970-08-24,-3051,1,0,1,1,Sales staff,1.0


In [6]:
# Apply same formula to DAYS_EMPLOYED
application_record_df['DAYS_EMPLOYED'] = application_record_df['DAYS_EMPLOYED'].apply(subtract_days)
application_record_df.head()

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,1990-02-05,2010-07-13,1,1,0,0,NaN,2.0
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,1990-02-05,2010-07-13,1,1,0,0,NaN,2.0
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,1964-03-04,2019-11-11,1,0,0,0,Security staff,2.0
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,1970-08-24,2014-08-12,1,0,1,1,Sales staff,1.0
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,1970-08-24,2014-08-12,1,0,1,1,Sales staff,1.0


### Transform Credit Record DataFrame

In [7]:
# Convert MONTHS_BALANCE column to actual month
current_date = datetime.now()
credit_record_df['MONTHS_BALANCE'] = credit_record_df['MONTHS_BALANCE'].abs()
credit_record_df['MONTHS_BALANCE'] = credit_record_df['MONTHS_BALANCE'].apply(lambda x: current_date - timedelta(weeks=x*4))
credit_record_df.head()

,ID,MONTHS_BALANCE,STATUS
0,5001711,2022-12-19 18:28:46.650473,X
1,5001711,2022-11-21 18:28:46.650473,0
2,5001711,2022-10-24 18:28:46.650473,0
3,5001711,2022-09-26 18:28:46.650473,0
4,5001712,2022-12-19 18:28:46.650473,C


### Create Database Connection

In [8]:
protocol = 'postgresql'
username = 'postgres'
password = 'postgres'
host = 'localhost'
port = 5432
database_name = 'creditcard_db'
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)
insp = inspect(engine)

In [9]:
# Confirm tables
insp.get_table_names()

['application_record', 'credit_record']

### Load DataFrames into database

In [10]:
application_record_df.to_sql(name='application_record', con=engine, if_exists='append', index=False)

In [11]:
credit_record_df.to_sql(name='credit_record', con=engine, if_exists='append', index=False)

### Test database with SQL query

In [12]:
pd.read_sql_query('SELECT * from application_record LIMIT 100', con=engine).head()

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,1990-02-01,2010-07-09,1,1,0,0,None,2.0
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,1990-02-01,2010-07-09,1,1,0,0,None,2.0
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,1964-02-29,2019-11-07,1,0,0,0,Security staff,2.0
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,1970-08-20,2014-08-08,1,0,1,1,Sales staff,1.0
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,1970-08-20,2014-08-08,1,0,1,1,Sales staff,1.0
